In [1]:
import json
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import time
from IPython.display import display
import ipywidgets as widgets
from scipy.optimize import fmin

In [4]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import plotly.graph_objects as go

In [29]:
import gmaps
import gmaps.datasets
with open('google_maps_key.txt', 'r') as fapi:
    api_key = fapi.read().strip()
    gmaps.configure(api_key=api_key)

In [30]:
DATA_DIR = os.path.join("data", "argentina")
JSON_RESULTS = os.path.join(DATA_DIR, "simulation_results.json")

# Generate data

In [7]:
!gen_dbs/generate_databases.py

In [8]:
!cd simulation && cmake -DCMAKE_BUILD_TYPE=Release . && make -j8

-- The C compiler identification is AppleClang 11.0.3.11030032
-- The CXX compiler identification is AppleClang 11.0.3.11030032
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - found
-- Found Threads: TRUE  
-- Boost version: 1.67.0
-- Found the following Boost libraries:
--   log_setup
-

In [9]:
parameters = {
    "incubation_period": 5.1,
    "duration_mild_infection": 10,
    "fraction_mild": 0.8,
    "fraction_severe": 0.15,
    "fraction_critical": 0.05,
    "CFR": 0.02,
    "time_ICU_death": 7,
    "duration_hospitalization": 11,
    "initial_new_cases":  10,
    "new_cases_rate":  pow(2, 1/5.),
    "home_contact_probability": 3.25e-2,
    "school_contact_probability": 3.57e-3,
    "neighbourhood_contact_probability": 9.6e-8,
    "inter_province_contact_probability": 1.23e-8,
}

In [10]:
pop_file = os.path.join(DATA_DIR, "fake_population")
pop_file+="_small"

In [11]:
def run_simulation(params):
    print("Running simulation...")
    !simulation/simulation \
        --days 150 \
        --population {pop_file} \
        --json {JSON_RESULTS} \
        --parameters "{json.dumps(params).replace('"', '\\"')}" \
        --seed 0 \
        --progress-only
run_simulation(parameters)

Running simulation...
[======================================================================] 100% 37.938s


# Show results

In [14]:
states = {
    "SUSCEPTIBLE": "Individuos susceptibles",
    "EXPOSED": "Individuos expuestos",
    "INFECTED_1": "Individuos infectados",
    "INFECTED_2": "Individuos infectados severamente",
    "INFECTED_3": "Individuos infectados críticamente",
    "RECOVERED": "Individuos recuperados",
    "DEAD": "Individuos muertos"
}
inf_sources = {
    "HOME_CONTACT": "Contacto en hogar",
    "SCHOOL_CONTACT": "Contacto en la escuela",
    "WORK_CONTACT": "Contacto en trabajo",
    "NEIGHBOURHOOD_CONTACT": "Contacto en vecindario",
    "INTER_PROVINCE_CONTACT": "Contacto interprovincial",
    "IMPORTED_CASE": "Contacto fuera del país",
}
with open(JSON_RESULTS) as f:
  sim_results = json.load(f)
sim_general = pd.DataFrame(sim_results["general"])
poblacion_total = sim_general[sim_general['day']==1][states].sum(axis=1)[0]
f"{poblacion_total} personas"

'3211030 personas'

In [13]:
fig = go.Figure()
for st,name in states.items():
    fig.add_trace(go.Scatter(x=sim_general["day"], y=sim_general[st]*1000/poblacion_total, name=name))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Casos pronosticados de COVID-19 por resultado clínico',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Casos por cada mil personas')


fig.show()

In [15]:
fig = go.Figure(data=[go.Pie(labels=list(inf_sources.values()), values=sim_general[inf_sources].sum())])
fig.update_layout(title="Fuentes de infecciones")
fig.show()

In [16]:
to_determine = ["home_contact_probability", "school_contact_probability", "neighbourhood_contact_probability", "inter_province_contact_probability"]

def contact_cost(x):
    cur_par = parameters.copy()
    for i,p in enumerate(to_determine):
        cur_par[p] = x[i]
    run_simulation(cur_par)
    with open(JSON_RESULTS) as f:
      sim_results = json.load(f)
    sim_general = pd.DataFrame(sim_results["general"])
    total = sim_general["HOME_CONTACT"].sum() \
        + sim_general["SCHOOL_CONTACT"].sum() \
        + (sim_general["NEIGHBOURHOOD_CONTACT"].sum()+sim_general["INTER_PROVINCE_CONTACT"].sum()+sim_general["IMPORTED_CASE"].sum())
    total = float(total)
    cost = (sim_general["HOME_CONTACT"].sum()/total-0.3)**2 \
        + (sim_general["SCHOOL_CONTACT"].sum()/total-0.3)**2 \
        + ((sim_general["NEIGHBOURHOOD_CONTACT"].sum()+sim_general["INTER_PROVINCE_CONTACT"].sum()+sim_general["IMPORTED_CASE"].sum())/total-0.3)**2
    print(f"f({x}) = {cost}")
    return cost
initial_values = np.array(list(map(lambda s: parameters[s], to_determine)))
#fmin(contact_cost, initial_values, full_output=True)

In [17]:
to_determine = ["home_contact_probability", "school_contact_probability", "neighbourhood_contact_probability", "inter_province_contact_probability"]

def contact_cost(x):
    cur_par = parameters.copy()
    for i,p in enumerate(to_determine):
        cur_par[p] = x[i]
    run_simulation(cur_par)
    with open(JSON_RESULTS) as f:
      sim_results = json.load(f)
    sim_general = pd.DataFrame(sim_results["general"])
    total = sim_general["HOME_CONTACT"].sum() \
        + sim_general["SCHOOL_CONTACT"].sum() \
        + (sim_general["NEIGHBOURHOOD_CONTACT"].sum()+sim_general["INTER_PROVINCE_CONTACT"].sum()+sim_general["IMPORTED_CASE"].sum())
    total = float(total)
    cost = (sim_general["HOME_CONTACT"].sum()/total-0.3)**2 \
        + (sim_general["SCHOOL_CONTACT"].sum()/total-0.3)**2 \
        + ((sim_general["NEIGHBOURHOOD_CONTACT"].sum()+sim_general["INTER_PROVINCE_CONTACT"].sum()+sim_general["IMPORTED_CASE"].sum())/total-0.3)**2
    print(f"f({x}) = {cost}")
    return cost
initial_values = np.array(list(map(lambda s: parameters[s], to_determine)))
#fmin(contact_cost, initial_values, full_output=True)

In [18]:
fig = go.Figure()
for st,name in inf_sources.items():
    fig.add_trace(go.Scatter(x=sim_general["day"], y=sim_general[st]*1000/poblacion_total, name=name))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Fuente de infecciones',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Infecciones por cada mil personas')


fig.show()

In [19]:
fig = go.Figure()
for st,name in inf_sources.items():
    fig.add_trace(go.Scatter(x=sim_general["day"], y=sim_general[st]*1000/poblacion_total, name=name))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Fuente de infecciones',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Infecciones por cada mil personas')


fig.show()

In [20]:
geodata = gpd.read_file(pop_file + '.gpkg', encoding='utf-8')
geodata.to_crs(epsg=4326,inplace=True)
geodata['centroid']=geodata.geometry.centroid.apply(lambda c: [c.y, c.x])
geodata

,cartodb_id,log_densid,viviendas,radio_id,prov_id,poblacion,hogares,frac_id,dpto_id,densidad,area,geometry,centroid
0,46185,8.21959338187,265.000000000000000,820070411,82,613,236,8200704,82007,3711.99230553551,0.165140,"MULTIPOLYGON (((-61.57636 -32.47684, -61.57502...","[-32.47871569032587, -61.576429889512255]"
1,45070,7.49990211117,260.000000000000000,820070214,82,638,232,8200702,82007,1806.86543596758,0.353098,"MULTIPOLYGON (((-61.76888 -32.56750, -61.76866...","[-32.56956488258713, -61.76543534967321]"
2,45220,7.91938719084,433.000000000000000,820070302,82,1162,396,8200703,82007,2749.08525182766,0.422686,"MULTIPOLYGON (((-61.59430 -32.77731, -61.59566...","[-32.77688433301194, -61.59958006371923]"
3,45700,7.87418728483,356.000000000000000,820070408,82,1082,335,8200704,82007,2627.54906028006,0.411791,"MULTIPOLYGON (((-61.57557 -32.46436, -61.57516...","[-32.468465629262646, -61.5769083397459]"
4,46671,7.88507640161,371.000000000000000,820070211,82,1339,392,8200702,82007,2656.32804243628,0.504079,"MULTIPOLYGON (((-61.53241 -32.68459, -61.53328...","[-32.67984423959501, -61.531749224854806]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,45875,0.125258780922,12.000000000000000,821330220,82,12,3,8213302,82133,0.133441728213397,89.926893,"MULTIPOLYGON (((-60.13733 -28.98139, -60.03908...","[-29.027811702666824, -60.0648454255209]"
3775,43524,0.316662676442,12.000000000000000,821330508,82,21,7,8213305,82133,0.372539503919527,56.369861,"MULTIPOLYGON (((-60.22616 -29.35387, -60.21761...","[-29.334223349718794, -60.16001346609598]"
3776,44551,7.91570571162,230.000000000000000,821330408,82,455,165,8213304,82133,2738.97948360646,0.166120,"MULTIPOLYGON (((-60.21008 -29.46155, -60.21026...","[-29.463052154596983, -60.21316023005173]"
3777,45375,0.24897084878,49.000000000000000,821330626,82,49,18,8213306,82133,0.282704640119065,173.325772,"MULTIPOLYGON (((-60.27195 -29.93792, -60.20037...","[-29.988831358442546, -60.167128219072126]"


In [21]:
sim_zones = pd.DataFrame(sim_results["by_zone"])
sim_zones.head()

,DEAD,EXPOSED,HOME_CONTACT,IMPORTED_CASE,INFECTED_1,INFECTED_2,INFECTED_3,INTER_PROVINCE_CONTACT,NEIGHBOURHOOD_CONTACT,RECOVERED,SCHOOL_CONTACT,SUSCEPTIBLE,UNDEFINED,WORK_CONTACT,day,zone
0,0,0,0,0,0,0,0,0,0,0,0,742,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,723,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,0,1265,0,0,1,2
3,0,0,0,0,0,0,0,0,0,0,0,1089,0,0,1,3
4,0,0,0,0,0,0,0,0,0,0,0,1187,0,0,1,4


In [22]:
contact_columns = list(filter(lambda c: c.endswith('CONTACT'), sim_zones.columns))+['IMPORTED_CASE']
sim_zones['CONTAGIOS'] = sim_zones[contact_columns].sum(axis=1)

In [31]:
class SeirExplorer(object):
    """
    Jupyter widget for exploring the SEIR simulation results.

    The user uses the slider to choose the day. This renders
    a heatmap of infected people in that day.
    """

    def __init__(self, locations, df, column, title, unit, min_display=10):
        self._df = df
        self._locations = locations
        self._heatmap = None
        self._slider = None
        self._column = column
        self._unit = unit
        self._min_display = min_display
        initial_day = 100

        title_widget = widgets.HTML(
            f'<h3>{title}</h3>'
        )

        map_figure = self._render_map(initial_day)
        controls = self._render_controls(initial_day)
        self._container = widgets.VBox([title_widget, controls, map_figure])

    def render(self):
        display(self._container)

    def _on_day_change(self, change):
        return self.update_day(self._slider.value)
    
    def update_day(self, day):
        self._slider.value = day
        self._heatmap.locations = self._locations_for_day(day)
        self._heatmap.weights = self._weights_for_day(day)
        self._total_box.value = self._total_text_for_day(day)
        return self._container

    def _render_map(self, initial_day):
        fig = gmaps.figure(map_type='HYBRID')
        self._heatmap = gmaps.heatmap_layer(
            self._locations_for_day(initial_day), weights=self._weights_for_day(initial_day),
            max_intensity=int(self._df.groupby('day').max()[self._column].max()),
            point_radius=5
        )
        fig.add_layer(self._heatmap)
        return fig

    def _render_controls(self, initial_day):
        self._slider = widgets.IntSlider(
            value=initial_day,
            min=min(self._df['day']),
            max=max(self._df['day']),
            description='Día',
            continuous_update=False
        )
        self._total_box = widgets.Label(
            value=self._total_text_for_day(initial_day)
        )
        self._slider.observe(self._on_day_change, names='value')
        controls = widgets.HBox(
            [self._slider, self._total_box],
            layout={'justify_content': 'space-between'}
        )
        return controls

    def _locations_for_day(self, day):
        self._locations['include'] = np.array(self._df[self._df['day'] == day][self._column]>self._min_display)
        ret = self._locations[self._locations['include']== True]['centroid']
        return ret if len(ret) else [self._locations['centroid'].iloc[0]]

    def _weights_for_day(self, day):
        ret = self._df[(self._df['day'] == day) & (self._df[self._column]>self._min_display)][self._column]
        return ret if len(ret) else np.array([0])

    def _total_for_day(self, day):
        return int(self._weights_for_day(day).sum())

    def _total_text_for_day(self, day):
        return f'{self._total_for_day(day)} {self._unit}'


#explorer = SeirExplorer(geodata, sim_zones, column='DEAD', unit="muertos", title="Muertos pronosticados de COVID-19 por ubicación geográfica")
explorer = SeirExplorer(geodata, sim_zones, column='CONTAGIOS', unit="contagios", title="Contagios pronosticados de COVID-19 sin cuarentena por ubicación geográfica")
#explorer = SeirExplorer(geodata, sim_zones, column='SUSCEPTIBLE', unit="susceptibles", title="Susceptibles pronosticados de COVID-19 sin cuarentena por ubicación geográfica")
explorer.render()

In [32]:
explorer._heatmap.point_radius = 15

In [33]:
for day in range(10, 87):
    explorer.update_day(day)
    time.sleep(0.01)
    if day<10:
        time.sleep(2)

In [34]:
explorer2 = SeirExplorer(geodata, sim_zones, column='SUSCEPTIBLE', unit="susceptibles", title="Susceptibles pronosticados de COVID-19 sin cuarentena por ubicación geográfica")
explorer2.render()

In [38]:
explorer2._heatmap.point_radius = 15

In [36]:
print(f"Tiempo total de la simulación: {sim_general['compute_time_ms'].sum()/1000/60:.2f} minutos")
fig = go.Figure()
fig.add_trace(go.Scatter(x=sim_general["day"], y=sim_general["compute_time_ms"], name=""))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


fig.update_layout(title='Performance de la simulación',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Duración (milisegundos)')


fig.show()

Tiempo total de la simulación: 0.58 minutos


In [37]:
for filename in ['simulation/zone_people', 'simulation/nearests_people', 'simulation/nearests']:
    with open(filename, 'r') as fin:
        asdf = fin.read()
    asdf = map(int, asdf.split())
    asdf = pd.DataFrame(asdf)
    asdf.iplot(kind="histogram", title=filename)

FileNotFoundError: [Errno 2] No such file or directory: 'simulation/zone_people'